# Повний цикл NLP-проєкту
## I. Перевірка фактів на достовірність
У межах цієї задачі ви побудуєте систему видобування фактів на правилах, а також інструменти для оцінювання якості роботи цієї системи.
### 1. Домен
Виберіть та проаналізуйте домен і напишіть SPARQL-запит для побудови бази даних.

Мій домен - це випускники Стенфорду. Цікавить:
- де вони народились
- ким працюють (occupation)
- Лінка на SPARQL Query - [link](http://dbpedia.org/snorql/?query=PREFIX+dbo%3A+%3Chttp%3A%2F%2Fdbpedia.org%2Fontology%2F%3E%0D%0APREFIX+res%3A++%3Chttp%3A%2F%2Fdbpedia.org%2Fresource%2F%3E%0D%0APREFIX+rdfs%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0D%0APREFIX+dbp%3A+%3Chttp%3A%2F%2Fdbpedia.org%2Fproperty%2F%3E%0D%0ASELECT+DISTINCT+%3Furi++%3Fname+%3Foccupation+%3Fcity+%0D%0AWHERE+%7B%3Furi+dbo%3AalmaMater+res%3AStanford_University+.%0D%0A++++++++%3Furi+rdfs%3Alabel+%3Fname.%0D%0A++++++++%3Furi+dbo%3Aoccupation+%3Focuri+.%0D%0A++++++++%3Focuri+rdfs%3Alabel+%3Foccupation+.%0D%0A++++++++%3Furi+dbo%3AbirthPlace+%3Fburi.%0D%0A++++++++%3Fburi+a+dbo%3ACity+.+%0D%0A++++++++%3Fburi+rdfs%3Alabel+%3Fcity+.++%0D%0A++++++++FILTER+%28lang%28%3Fcity%29+%3D+%27en%27%29%0D%0A++++++++FILTER+%28lang%28%3Fname%29+%3D+%27en%27%29%0D%0A++++++++FILTER+%28lang%28%3Foccupation%29+%3D+%27en%27%29%0D%0A%7D)

In [ ]:
from IPython.display import FileLink
FileLink('stanford_alumni.csv', result_html_prefix="База даних з DBPedia складається з 155 рядків: ")

### 2. Видобування фактів
2.1. Напишіть програму, яка шукає статті у Вікіпедії про сутності, що належать до вашого домена, та витягає тексти цих статей.

In [2]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [3]:
wiki_links = []

with open('stanford_alumni.csv', "r") as f:
    reader = csv.reader(f)
    first_line = next(reader)
    for row in reader:
        url = row[0].replace('http://dbpedia.org/resource/', 'http://en.wikipedia.org/wiki/')
        if url not in wiki_links:
            wiki_links.append(url)

print(len(wiki_links))
print(wiki_links[:3])

97
['http://en.wikipedia.org/wiki/William_Ouchi', 'http://en.wikipedia.org/wiki/Mark_T._Carleton', 'http://en.wikipedia.org/wiki/David_Allais']


In [4]:
def get_wiki_item_text(item_url):
    source_code = requests.get(item_url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    for item_name in soup.findAll('title'):
        title = soup.title.string 
        title = title[:-12]
        for item_name in soup.findAll('div',{'class':'mw-content-ltr'}):
            wiki_text = [x.text for x in soup.findAll('p')]
            wiki_text =''.join(wiki_text)

    return wiki_text, title
get_wiki_item_text("http://en.wikipedia.org/wiki/William_Ouchi")

('William G. "Bill" Ouchi (born 1943) is an American professor and author in the field of business management. He is the Distinguished Professor of Management and Organizations, Stanford and Betty Sigoloff Chair in Corporate Renewal at the UCLA Anderson School of Management.\nHe was born and raised in Honolulu, Hawaii. He earned a B.A. from Williams College in 1965, and an MBA from Stanford University and a Ph.D. in Business Administration from the University of Chicago. He was a Stanford business school professor for 8 years and has been a faculty member of the Anderson School of Management at University of California, Los Angeles for many years.\nOuchi first came to prominence for his studies of the differences between Japanese and American companies and management styles.\nHis first book in 1981 summarized his observations. Theory Z: How American Management Can Meet the Japanese Challenge and was a New York Times best-seller for over five months.\nHis second book, The M Form Society

In [5]:
f_name = "wikipedia_texts.txt"
result = open(f_name, "w+")
not_found = []

for link in wiki_links:
    text, title = get_wiki_item_text(link)
    if "Other reasons this message may be displayed" in text:
        not_found.append(title)
    else:
        result.write("TITLE: {}\n{}\n".format(title, text))

print("Not found alumni:", not_found)

Not found alumni: ['Mark T. Carleton', 'Kyle Wong', 'Anand Chandrasekaran', 'Mercy Malick']


In [6]:
# знайшла urls з інформацією для відсутніх у вікіпедії  
not_found_urls = ["https://wikivisually.com/wiki/Mark_T._Carleton", 
"https://www.huffpost.com/entry/incubating-an-entrepreneur-pixlees-kyle-wong-on-starting_b_59d41a79e4b08c2a000ddcce",
"https://en.everybodywiki.com/Anand_Chandrasekaran", "https://peoplepill.com/people/mercy-malick/"]

### Висновки по DBPedia
1. Витягати випускників можна по-різному: dbo:almaMater - дає 1135 рядків, dbo:education - 145 та ін. Тому обрала dbo:almaMater. Після додавання місця народження та професії вона зменшилась, оскільки не всі ці поля є в кожного випускника. Хотіла взяти національність, але отримувала, ще меншу БД. Дехто з випускників має кілька професій, а тому також отримала дублікати, які відкинула при пошуці у Вікіпедії (тому не 155, а 97 текстів)

In [7]:
alumni = pd.read_csv('stanford_alumni.csv', header=0) # 155 entries with duplicates,
alma = pd.read_csv('stanf_alma.csv', header=0) # dbo:almaMater - 1135 entries 
edu = pd.read_csv('stanf_edu.csv', header=0) # dbo:education - 145 entries